# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [8]:
r.set('greetings', 'hello world')
greetings = r.get('greetings')
print(greetings)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [13]:
r.set('connections', 0)
connections = 0
r.incr('connections')
print(r.get('connections'))

b'1'


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [18]:
r.rpush('colors', 'red', 'black', 'blue')


[b'red', b'black', b'blue', b'red', b'black', b'blue', b'red', b'black', b'blue']


**Q** : Display the second element of the list with `LINDEX`

In [23]:
color2 = r.lindex('colors', 1)
print(color2)

b'black'


**Q** : Display the full list with `LRANGE`

In [19]:
colors=r.lrange('colors', 0, -1)
print(colors)

[b'red', b'black', b'blue', b'red', b'black', b'blue', b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [24]:
r.sadd('nosql', 'redis', 'mongodb', 'orientdb')

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [27]:
r.sismember('nosql','mysql')

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [28]:
r.sadd('nosql', 'hbase')
elements = r.smembers('nosql')
print(elements)

r.sadd('nosql', 'redis')

{b'orientdb', b'mongodb', b'hbase', b'redis'}


0

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [36]:
top14_data = {
        'Agen': 10,
        'Bordeaux': 33,
        'Brive': 32,
        'Castres': 29,
        'Clermont': 38,
        'Grenoble': 24,
        'La Rochelle': 26,
        'Montpellier': 32,
        'Oyonnax': 14,
        'Pau': 20,
        'Racing': 40,
        'Stade Français': 22,
        'Toulon': 36,
        'Toulouse': 36
    }

r.zadd('top14', top14_data)

14

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [40]:
toulon_score = r.zscore('top14','Toulon')
toulon_ranking = r.zrank('top14', 'Toulon')
print(toulon_score)
print(toulon_ranking)

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [41]:
toulon_ranking = r.zrevrank('top14', 'Toulon')
print(toulon_ranking)

3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [67]:
r.zrange('top14', 11, 14, withscores=True)

[(b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [53]:
dico = {'id' : 1, 'name': "Jean", 'age':22}
r.hmset('user:1', dico)

C:\Users\dbourhani\.conda\envs\nosql\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  


True

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [61]:
r.hmset('user:1', {'city' : 'Lyon'})

C:\Users\dbourhani\.conda\envs\nosql\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  """Entry point for launching an IPython kernel.


[b'id', b'name', b'age', b'city']

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [88]:
import json
import requests

url="PUT_http://my-api.fr/user/10"
data = {"id": 10, "name": "jean"}

r.set(url, json.dumps(data))        

True

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [93]:
url="GET_http://my-api.fr/user?size=2"
data = [{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]

r.set(url, json.dumps(data))        

True

**Q** : Create a set of cache request keys.

In [102]:
for key in r.keys(pattern="*http://*"):
    r.sadd('cached_keys', key)

In [103]:
print(r.smembers('cached_keys'))

{b'PUT_http://my-api.fr/user/10', b'GET_http://my-api.fr/user?size=2'}


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [104]:
r.keys(pattern="GET_http://my-api.fr/user*")

[b'GET_http://my-api.fr/user?size=2']

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [106]:
r.delete("PUT_http://my-api.fr/user/10")

1

In [108]:
r.keys(pattern="*http://*")

[b'GET_http://my-api.fr/user?size=2']

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [109]:
!redis-cli flushall

'redis-cli' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.
